In [2]:
using Gen, StatsFuns, StatsPlots

In [3]:
begin
	ROCK = 1
	PAPER = 2
	SCISSORS = 3
	
	"returns the reward of player a"
	function reward(choice_a, choice_b)
		reward = 0
		if choice_a != choice_b 
			reward = 1
			if choice_a < choice_b 
				choice_a, choice_b = choice_b, choice_a
				reward = -reward
			end
			if choice_a == SCISSORS && choice_b == ROCK
				reward = -reward
			end
		end
		reward
	end
	
	@assert reward(ROCK, ROCK)==0
	@assert reward(ROCK, PAPER)==-1
	@assert reward(PAPER, ROCK)==1
	@assert reward(ROCK, SCISSORS)==1
	@assert reward(PAPER, SCISSORS)==-1
end

In [4]:
"draw preference vector (Gen does not have Dirichlet)"
@gen function pref(alpha)
	pref = Vector{Float64}(undef, length(alpha))
	for i = eachindex(alpha)
		pref[i] = @trace(gamma(alpha[i], 1), i)
	end
	return pref ./ sum(pref)
end;

In [5]:
@gen function static_game(n, gamma=1, alpha_a=[1,1,1], alpha_b=[1,1,1])
	pref_a = @trace(pref(alpha_a), :pref_a)
	pref_b = @trace(pref(alpha_b), :pref_b)
	for i = 1:n
		@trace(categorical(pref_a), (:choice_a, n-i+1))
		@trace(categorical(pref_b), (:choice_b, n-i+1))
	end
	choice_a = @trace(categorical(pref_a), (:choice_a, 0))
	choice_b = @trace(categorical(pref_b), (:choice_b, 0))
	score = reward(choice_a, choice_b)
	p_a = logistic(gamma*score)
	@trace(bernoulli(p_a), :win_a)
	return score
end;

In [6]:
static_game(5)

1

In [7]:
@gen function markov_game(n, m_a=1, m_b=1, gamma=1, alpha_a=[1,1,1], alpha_b=[1,1,1])
	# Last preferences (m is the stubbornness of each player)
	for i = 1:m_a
		@trace(pref(alpha_a), (:pref_a_last, i))
	end
	for i = 1:m_b
		@trace(pref(alpha_b), (:prev_b_last, i))
	end
	
	# Current preferences
	pref_a = @trace(pref(alpha_a), :pref_a)
	pref_b = @trace(pref(alpha_b), :pref_b)

	# Choices (TODO: theory of mind)
	choice_a = @trace(categorical(pref_a), (:choice_a, 0))
	choice_b = @trace(categorical(pref_b), (:choice_b, 0))
	
	score = reward(choice_a, choice_b)
	p_a = logistic(gamma*score)
	@trace(bernoulli(p_a), :win_a)
	
	return score
end;

In [8]:
markov_game(25)

-1